In [1]:
import ee

In [2]:
# Trigger the authentication flow. if you want to user json, please comment this
ee.Authenticate()
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1Adeu5BVql01Z1FNfJSVId2h4xa5XgE-SpeD_tJTnPj9Gywfk_pwZl47Ofog

Successfully saved authorization token.


In [43]:
import threading

In [44]:
def retrieve(f):
    def wrap(obj, *args):
        f(obj.getInfo(), *args)
    return wrap


def evaluate(obj, callback, args):
    args.insert(0, obj)
    callback = retrieve(callback)
    thd = threading.Thread(target=callback, args=args)
    thd.start()
    

# EE object
eeobj = ee.List([1,2,3,4,5,6])
    
# use case 1: printing values
def callback1(obj, n):
    print([i*n for i in obj])

evaluate(eeobj, callback1, [5])

# use case 2: filling a container
container = []
def callback2(obj, cont):
    """ filling a container """
    for item in obj:
        cont.append(item)
        
evaluate(eeobj, callback2, [container])




[5, 10, 15, 20, 25, 30]


In [45]:
# wait 2 sec and...
print(container)

[1, 2, 3, 4, 5, 6]


In [14]:
evaluate(eeobj, callback2, [container])

In [16]:
import asyncio

In [21]:
# syncrhonous
import time

def count():
    print("One")
    time.sleep(1)
    print("Two")

def main():
    for _ in range(3):
        count()

if __name__ == "__main__":
    s = time.perf_counter()
    main()
    elapsed = time.perf_counter() - s
    print(f" executed in {elapsed:0.2f} seconds.")

One
Two
One
Two
One
Two
 executed in 3.00 seconds.


In [26]:
for i in range(3):
    print(i)

0
1
2


In [23]:
# asynchronous
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

if __name__ == "__main__":
    import time
    s = time.perf_counter()
    
    # In your case, jupyter (IPython ≥ 7.0) is already running an event loop:
    #asyncio.run(main())
    
    await main()
    elapsed = time.perf_counter() - s
    print(f" executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
 executed in 1.00 seconds.


In [24]:
async def g():
    # Pause here and come back to g() when f() is ready
    r = await f()
    return r

In [25]:
async def f(x):
    y = await z(x)  # OK - `await` and `return` allowed in coroutines
    return y

async def g(x):
    yield x  # OK - this is an async generator

async def m(x):
    yield from gen(x)  # No - SyntaxError

def m(x):
    y = await z(x)  # Still no - SyntaxError (no `async def` here)
    return y

SyntaxError: 'yield from' inside async function (1355037374.py, line 9)

In [27]:
eeobj = ee.List([1,2,3,4,5,6])

In [28]:
async def fetch_image(collection_id, geometry):
    # Perform an Earth Engine task asynchronously
    image = ee.ImageCollection(collection_id).filterBounds(geometry).first()
    await asyncio.sleep(1)  # Simulate some async work
    return image.getInfo()

In [29]:
async def main():
    # Define your geometry or other parameters
    geometry = ee.Geometry.Point(-122.082, 37.42)

    # Create a list of tasks to run concurrently
    tasks = [
        fetch_image('LANDSAT/LC08/C01/T1_TOA', geometry),
        fetch_image('COPERNICUS/S2', geometry),
    ]

    # Gather the results of the tasks
    results = await asyncio.gather(*tasks)

    # Process the results as needed
    for result in results:
        print(result)

In [32]:
if __name__ == "__main__":
    import time
    s = time.perf_counter()
    
    # In your case, jupyter (IPython ≥ 7.0) is already running an event loop:
    #asyncio.run(main())
    
    await main()
    elapsed = time.perf_counter() - s
    print(f" executed in {elapsed:0.2f} seconds.")

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:

In [33]:
def fetch_image(collection_id, geometry):
    # Perform an Earth Engine task synchronously
    image = ee.ImageCollection(collection_id).filterBounds(geometry).first()
    #await asyncio.sleep(1)  # Simulate some async work
    return image.getInfo()

In [34]:
def main():
    # Define your geometry or other parameters
    geometry = ee.Geometry.Point(-122.082, 37.42)

    # run sync a list of tasks
    print(fetch_image('LANDSAT/LC08/C01/T1_TOA', geometry))
    print(fetch_image('COPERNICUS/S2', geometry))
    

In [35]:
if __name__ == "__main__":
    s = time.perf_counter()
    main()
    elapsed = time.perf_counter() - s
    print(f" executed in {elapsed:0.2f} seconds.")

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 494085, 0, -30, 4257915]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [7511, 7351], 'crs': 'EPSG:

In [37]:
# EE object
eeobj = ee.List([1, 2, 3, 4, 5, 6])

# Async function to retrieve data and invoke the callback
async def evaluate_async(obj, callback, args):
    try:
        data = await asyncio.to_thread(obj.getInfo)
        callback(data, *args)
    except Exception as e:
        print(f"An error occurred: {e}")

# Use case 1: Printing values
def callback1(obj, n):
    print([i * n for i in obj])

async def use_case1():
    await evaluate_async(eeobj, callback1, [5])

# Use case 2: Filling a container
container = []

def callback2(obj, cont):
    """ Filling a container """
    for item in obj:
        cont.append(item)

async def use_case2():
    await evaluate_async(eeobj, callback2, [container])

async def main():
    await asyncio.gather(use_case1(), use_case2())
    # Wait for both use cases to complete

if __name__ == '__main__':
    #asyncio.run(main())
    await main()
    # Wait for the main async function to complete
    print(container)  # Print the container after use_case2 has filled it

[5, 10, 15, 20, 25, 30]
[1, 2, 3, 4, 5, 6]


In [38]:
# Function to retrieve data and invoke the callback
def evaluate_sync(obj, callback, args):
    try:
        data = obj.getInfo()
        callback(data, *args)
    except Exception as e:
        print(f"An error occurred: {e}")

# Use case 1: Printing values
def callback1(obj, n):
    print([i * n for i in obj])

def use_case1():
    evaluate_sync(eeobj, callback1, [5])

# Use case 2: Filling a container
container = []

def callback2(obj, cont):
    """ Filling a container """
    for item in obj:
        cont.append(item)

def use_case2():
    evaluate_sync(eeobj, callback2, [container])

def main():
    use_case1()
    use_case2()
    # No need to wait since everything is synchronous

if __name__ == '__main__':
    main()
    print(container)  # Print the container after use_case2 has filled it

[5, 10, 15, 20, 25, 30]
[1, 2, 3, 4, 5, 6]
